In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from linearmodels import PanelOLS
import statsmodels.api as sm
import econtools as econ
import econtools.metrics as mt
import math
from statsmodels.stats.outliers_influence import variance_inflation_factor

from auxiliary.prepare import *
from auxiliary.table2 import *
from auxiliary.table3 import *
from auxiliary.table4 import *
from auxiliary.table6 import *
from auxiliary.table7 import *
from auxiliary.table_formula import *

In [2]:
df_auth = pd.read_stata('data/Authority.dta')
df_IE = pd.read_stata('data/IE.dta')
df_auth = basic_setting(df_auth)

In [171]:
def extension_setting_IE(IE, Authority):
    df_IE = IE
    df_auth = Authority
    ctrl_exp_vlunt_list = df_IE.loc[df_IE['adopters_vlunt']==1,'experience'].unique()
    ctrl_pop_vlunt_list = df_IE.loc[df_IE['adopters_vlunt']==1,'population'].unique()
    df_IE['ctrl_exp_adopters_vlunt'] =0
    df_IE['ctrl_pop_adopters_vlunt'] =0
    for i in df_IE['experience'].unique():
        for j in range(len(ctrl_exp_vlunt_list)):
            if i == ctrl_exp_vlunt_list[j]:
                df_IE.loc[df_IE['experience']==i, 'ctrl_exp_adopters_vlunt'] =1
    for i in df_IE['population'].unique():
        for j in range(len(ctrl_pop_vlunt_list)):
            if i == ctrl_pop_vlunt_list[j]:
                df_IE.loc[df_IE['population']==i, 'ctrl_pop_adopters_vlunt'] =1
    
    work_dum = pd.get_dummies(df_IE['work_category']).rename(columns=lambda x: 'work_dum_' + str(x))
    year_dum = pd.get_dummies(df_IE['year']).rename(columns=lambda x: 'year_dum_' + str(x))
    df_IE = pd.concat([df_IE,work_dum],axis = 1)
    df_IE = pd.concat([df_IE,year_dum],axis = 1)

    return(df_IE)

df_fuc = extension_setting_IE(df_IE, df_auth)

In [76]:
ctrl_exp_vlunt_list = df_IE.loc[df_IE['adopters_vlunt']==1,'experience'].unique()
ctrl_pop_vlunt_list = df_IE.loc[df_IE['adopters_vlunt']==1,'population'].unique()

In [78]:
df_IE['ctrl_exp_adopters_vlunt'] =0
df_IE['ctrl_pop_adopters_vlunt'] =0
for i in df_IE['experience'].unique():
    for j in range(len(ctrl_exp_vlunt_list)):
        if i == ctrl_exp_vlunt_list[j]:
            df_IE.loc[df_IE['experience']==i, 'ctrl_exp_adopters_vlunt'] =1
for i in df_IE['population'].unique():
    for j in range(len(ctrl_pop_vlunt_list)):
        if i == ctrl_pop_vlunt_list[j]:
            df_IE.loc[df_IE['population']==i, 'ctrl_pop_adopters_vlunt'] =1

In [ ]:
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

In [122]:
#voluntary group들을 turin으로 대체하고
#ctrl group을

df= df_IE
work_dum = pd.get_dummies(df['work_category']).rename(columns=lambda x: 'work_dum_' + str(x))
year_dum = pd.get_dummies(df['year']).rename(columns=lambda x: 'year_dum_' + str(x))
#auth_dum = pd.get_dummies(df1['authority_code']).rename(columns=lambda x: 'auth_dum_' + str(x))
df = pd.concat([df,work_dum],axis = 1)
df = pd.concat([df,year_dum],axis = 1)

work_list = df['work_category'].unique()
year_list = df['year'].unique()
df = df[(df['adopters_vlunt']==1)&(df['ctrl_exp_adopters_vlunt']==1)]
outcome = ['discount', 'days_to_award']

#ctrl_exp
#col1 iyear
for o in outcome:
    
    reg_col = []
    for j in year_list:
        reg_col.append('year_dum_' + str(j))
    
    exog_var = ['fpsb_auction']
    exog = exog_var + reg_col 


    #check multicollinearity
    X = df.loc[:,exog]
    vif = calc_vif(X)

    #delete from col list
    for i in range(len(vif)):
        if np.isnan(vif.loc[i, 'VIF']) == True:
            reg_col.remove(vif.loc[i, 'variables'])

    exog = exog_var + reg_col
    exog.remove('year_dum_2003.0')
    
    if o == 'discount':
        fe_reg_discount = mt.reg(df, o, exog, fe_name = 'authority_code', cluster = 'auth_anno', addcons = True, check_colinear = True)
    else :
        exog.remove('year_dum_2010.0')
        fe_reg_award = mt.reg(df, o, exog, fe_name = 'authority_code', cluster = 'auth_anno',addcons = True, check_colinear = True)

fe_reg_1 = (fe_reg_discount, fe_reg_award )

print(fe_reg_1)

#col2 iyear, iwork, reserve_price, municipality
for o in outcome:
    reg_col = []
    for i in work_list:
        reg_col.append('work_dum_' + i)
    for j in year_list:
        reg_col.append('year_dum_' + str(j))
    
    exog_var = ['fpsb_auction','reserve_price','municipality']
    exog = exog_var + reg_col 

    #check multicollinearity

    #exog = exog_var + reg_col
    exog.remove('work_dum_OG03')
    exog.remove('municipality')
    exog.remove('year_dum_2003.0')

    if o == 'discount':
        fe_reg_discount = mt.reg(df, o, exog, fe_name = 'authority_code', cluster = 'auth_anno', addcons = True, check_colinear = True)
    else :
        exog.remove('year_dum_2010.0')
        fe_reg_award = mt.reg(df, o, exog, fe_name = 'authority_code', cluster = 'auth_anno', addcons = True, check_colinear = True)

fe_reg_2 = (fe_reg_discount, fe_reg_award )

print(fe_reg_2)

(=======================================================
Dependent variable:	discount
N:			657
R-squared:		0.5519
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  96
Fixed effects by:	authority_code
  No. of FE:		  24
                 coeff    se      t   p>t CI_low CI_high
fpsb_auction     9.342 1.620  5.766 0.000  6.125  12.558
year_dum_2005.0 10.920 1.472  7.417 0.000  7.997  13.843
year_dum_2006.0 13.119 2.016  6.509 0.000  9.117  17.121
year_dum_2007.0 15.742 1.353 11.638 0.000 13.056  18.427
year_dum_2009.0 16.013 1.499 10.679 0.000 13.036  18.989
year_dum_2008.0 14.715 1.461 10.071 0.000 11.815  17.616
year_dum_2010.0 19.635 2.861  6.862 0.000 13.954  25.315
, =======================================================
Dependent variable:	days_to_award
N:			379
R-squared:		0.5067
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  60
Fixed effects by:	authority_code
  No. of FE:		  20
      

In [123]:
ctrl_exp_forced_list = df_IE.loc[df_IE['forcedfp_strict'],'experience'].unique()
ctrl_pop_forced_list = df_IE.loc[df_IE['forcedfp_strict'],'population'].unique()

In [134]:
df_auth['ctrl_exp_forcedfp_co'] = df_auth['ctrl_exp_turin_co_sample']
df_auth['ctrl_exp_forcedfp_pr'] = df_auth['ctrl_exp_turin_pr_sample']
for i in ctrl_exp_forced_list:
    for j in range(len(df_auth)):
        if df_auth.loc[j, 'experience'] == i:
            df_auth.loc[j, 'ctrl_exp_forcedfp_co'] = 0
            df_auth.loc[j, 'ctrl_exp_forcedfp_pr'] = 0
            
df_auth['ctrl_pop_forcedfp_co'] = df_auth['ctrl_pop_turin_co_sample']
df_auth['ctrl_pop_forcedfp_pr'] = df_auth['ctrl_pop_turin_pr_sample']
for i in ctrl_pop_forced_list:
    for j in range(len(df_auth)):
        if df_auth.loc[j, 'population'] == i:
            df_auth.loc[j, 'ctrl_pop_forcedfp_co'] = 0
            df_auth.loc[j, 'ctrl_pop_forcedfp_pr'] = 0

In [142]:
df_auth['ctrl_pop_forcedfp_co'].value_counts() #원래 거랑 똑같음

0.0    12202
1.0     3925
Name: ctrl_pop_forcedfp_co, dtype: int64

In [149]:
#main table
#col1
df =df_auth
work_list = df['work_category'].unique()
year_list = df['year'].unique()
df = df[(df['turin_co_sample']==1)&(df['ctrl_exp_forcedfp_co']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & 
               (df['pre_experience'].isnull()==False)&(df['missing']==0)]
outcome = ['discount', 'delay_ratio', 'overrun_ratio', 'days_to_award']

#iteration
for o in outcome:
    
    #make a column list
    reg_col = []
    for j in year_list:
        reg_col.append(j)
    exog_var = ['fpsb_auction']
    exog = exog_var + reg_col 


    #check multicollinearity
    X = df.loc[:,exog]
    vif = calc_vif(X)

    #delete from col list
    for i in range(len(vif)):
        if np.isnan(vif.loc[i, 'VIF']) == True:
            reg_col.remove(vif.loc[i, 'variables'])

    exog = exog_var + reg_col
    exog.remove(2000)

    if o == 'discount':
        fe_reg_discount = mt.reg(df, o, exog, fe_name = 'authority_code', cluster = 'auth_anno',check_colinear = True)
    elif o == 'delay_ratio':
        fe_reg_delay = mt.reg(df, o, exog, fe_name = 'authority_code', cluster = 'auth_anno',check_colinear = True)
    elif o == 'overrun_ratio':
        fe_reg_overrun = mt.reg(df, o, exog, fe_name = 'authority_code', cluster = 'auth_anno',check_colinear = True)
    else :
        fe_reg_award = mt.reg(df, o, exog, fe_name = 'authority_code', cluster = 'auth_anno',check_colinear = True)

fe_reg = (fe_reg_discount, fe_reg_delay, fe_reg_overrun, fe_reg_award )
print(fe_reg)

(=======================================================
Dependent variable:	discount
N:			1262
R-squared:		0.5047
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  101
Fixed effects by:	authority_code
  No. of FE:		  15
              coeff    se      t   p>t CI_low CI_high
fpsb_auction 13.095 1.609  8.137 0.000  9.902  16.288
2006.0        4.024 1.281  3.140 0.002  1.482   6.566
2003.0       -0.243 1.111 -0.219 0.827 -2.446   1.961
2001.0        2.181 1.144  1.906 0.060 -0.089   4.450
2002.0        0.583 1.140  0.512 0.610 -1.678   2.844
2005.0        3.764 1.155  3.259 0.002  1.473   6.056
2004.0        1.580 1.034  1.528 0.130 -0.472   3.631
, =======================================================
Dependent variable:	delay_ratio
N:			1110
R-squared:		0.0853
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  99
Fixed effects by:	authority_code
  No. of FE:		  15
               coeff     se  

In [164]:
#main table
#col 2
df = df_auth
work_dum = pd.get_dummies(df['work_category']).rename(columns=lambda x: 'work_dum_' + str(x))
year_dum = pd.get_dummies(df['year']).rename(columns=lambda x: 'year_dum_' + str(x))
#auth_dum = pd.get_dummies(df1['authority_code']).rename(columns=lambda x: 'auth_dum_' + str(x))
df = pd.concat([df,work_dum],axis = 1)
df = pd.concat([df,year_dum],axis = 1)

work_list = df['work_category'].unique()
year_list = df['year'].unique()

df = df[(df['turin_co_sample']==1)&(df['ctrl_exp_forcedfp_co']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) &
               (df['pre_experience'].isnull()==False)&(df['missing']==0)]
outcome = ['discount', 'delay_ratio', 'overrun_ratio', 'days_to_award']

#iteration
for o in outcome:

    #make a column list
    reg_col = []
    for i in work_list:
        reg_col.append('work_dum_' + i)
    for j in year_list:
        reg_col.append('year_dum_'+ str(j))
    exog_var = ['fpsb_auction','reserve_price','municipality']
    exog = exog_var + reg_col 
    
    X = df.loc[:,exog]
    vif = calc_vif(X)
    #delete from col list
    for i in range(len(vif)):
        if np.isnan(vif.loc[i, 'VIF']) == True:
            reg_col.remove(vif.loc[i, 'variables'])
        elif vif.loc[i,'VIF'] > 10:
            for j in exog:
                if vif.loc[i,'variables'] == j and vif.loc[i,'variables'] != 'fpsb_auction':
                    exog.remove(vif.loc[i,'variables'])
   
    #exog.remove('year_dum_2000.0')
    #exog.remove('work_dum_OG01')
    exog.remove('municipality')
    exog.remove('work_dum_OS03')
    exog.remove('work_dum_OS04')
    exog.remove('work_dum_OS33')
    exog.remove('work_dum_OG13')
    exog.remove('work_dum_OS09')
    exog.remove('work_dum_OS13')
    exog.remove('work_dum_OS32')
    exog.remove('work_dum_OG09')
    exog.remove('work_dum_OG21')
    exog.remove('work_dum_OG18')
    exog.remove('work_dum_OG24')
    exog.remove('work_dum_OG05')
    exog.remove('year_dum_2008.0')
    exog.remove('year_dum_2009.0')
    exog.remove('year_dum_2007.0')
    exog.remove('year_dum_2010.0')
    #check multicollinearity
    if o == 'discount':

        fe_reg_discount = mt.reg(df, o, exog, fe_name = 'authority_code', cluster = 'auth_anno',addcons= True, check_colinear = True)

    elif o == 'delay_ratio':
        fe_reg_delay = mt.reg(df, o, exog, fe_name = 'authority_code', cluster = 'auth_anno',addcons = True, check_colinear =  True)

    elif o == 'overrun_ratio':
        fe_reg_overrun = mt.reg(df, o, exog, fe_name = 'authority_code', cluster = 'auth_anno',addcons = True, check_colinear = True)

    else :
        fe_reg_award = mt.reg(df, o, exog, fe_name = 'authority_code', cluster = 'auth_anno',addcons = True, check_colinear = True)


fe_reg = (fe_reg_discount, fe_reg_delay, fe_reg_overrun, fe_reg_award )
print(fe_reg)

(=======================================================
Dependent variable:	discount
N:			1262
R-squared:		0.4872
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  101
Fixed effects by:	authority_code
  No. of FE:		  15
               coeff    se     t   p>t CI_low CI_high
fpsb_auction  13.894 1.736 8.004 0.000 10.450  17.338
reserve_price  0.000 0.000 5.395 0.000  0.000   0.000
, =======================================================
Dependent variable:	delay_ratio
N:			1110
R-squared:		0.0756
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  99
Fixed effects by:	authority_code
  No. of FE:		  15
               coeff    se      t   p>t CI_low CI_high
fpsb_auction  14.879 9.946  1.496 0.138 -4.859  34.617
reserve_price -0.000 0.000 -1.918 0.058 -0.000   0.000
, =======================================================
Dependent variable:	overrun_ratio
N:			1092
R-squared:		0.1176
Estimation me

In [178]:
len(df_auth[(df_auth['ctrl_exp_forcedfp_pr']==1)&(df_auth['ctrl_pop_forcedfp_pr']==1)]) #값이 같아서 따로 돌리지 않음

2097

In [179]:
len(df_auth[df_auth['ctrl_pop_exp_turin_pr_sample']==1])

2501

In [236]:
#main table turin_pr
#main table
#col1(exp)
df = df_auth
df = df[(df['turin_pr_sample']==1)&(df['ctrl_pop_forcedfp_pr']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
df = df.reset_index()
work_dum = pd.get_dummies(df['work_category']).rename(columns=lambda x: 'work_dum_' + str(x))
year_dum = pd.get_dummies(df['year']).rename(columns=lambda x: 'year_dum_' + str(x))
df = pd.concat([df,work_dum],axis = 1)
df = pd.concat([df,year_dum],axis = 1)

year_list = df['year'].unique()
work_list = df['work_category'].unique()
outcome = ['discount', 'delay_ratio', 'overrun_ratio', 'days_to_award']

#iteration
for o in outcome:
    
    #make a column list
    reg_col = []
    for j in year_list:
        reg_col.append('year_dum_'+str(j))
    exog_var = ['fpsb_auction']
    exog = exog_var + reg_col 


    #check multicollinearity
    '''#X = df.loc[:,exog]
    #vif = calc_vif(X)

    #delete from col list
    for i in range(len(vif)):
        if np.isnan(vif.loc[i, 'VIF']) == True:
            exog.remove(vif.loc[i, 'variables'])

    exog = exog_var + reg_col'''
    exog.remove('year_dum_2000.0')
    exog.remove('year_dum_2006.0')
    exog.remove('year_dum_2005.0')
    exog.remove('year_dum_2004.0')
    exog.remove('year_dum_2003.0')
    exog.remove('year_dum_2002.0')
    exog.remove('year_dum_2001.0')
    
    
    if o == 'discount':
        fe_reg_discount = mt.reg(df, o, exog, fe_name = 'authority_code', cluster = 'auth_anno', addcons = True, check_colinear = True)
    if o == 'delay_ratio':
        fe_reg_delay = mt.reg(df, o, exog, fe_name = 'authority_code', cluster = 'auth_anno', addcons = True, check_colinear = True)
    elif o == 'overrun_ratio':
        fe_reg_overrun = mt.reg(df, o, exog, fe_name = 'authority_code', cluster = 'auth_anno',addcons = True, check_colinear = True)
    else :
        fe_reg_award = sm.OLS(df[o],df[exog])

fe_reg = (fe_reg_discount, fe_reg_delay, fe_reg_overrun, fe_reg_award )

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1830: RuntimeWarning: invalid value encountered in float_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\base\model.py:1362: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:               discount   R-squared (uncentered):                   0.000
Model:                            OLS   Adj. R-squared (uncentered):              0.000
Method:                 Least Squares   F-statistic:                                nan
Date:                Tue, 14 Jul 2020   Prob (F-statistic):                         nan
Time:                        22:33:56   Log-Likelihood:                         -2021.2
No. Observations:                 506   AIC:                                      4042.
Df Residuals:                     506   BIC:                                      4042.
Df Model:                           0                                                  
Covariance Type:            nonrobust                                                  
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
fpsb_auction          0          0        nan        nan           0           0
==============================================================================
Omnibus:                       77.772   Durbin-Watson:                   0.264
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              201.341
Skew:                           0.770   Prob(JB):                     1.90e-44
Kurtosis:                       5.679   Cond. No.                          nan
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is      0. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [249]:
df = df_auth
df = df[(df['turin_pr_sample']==1)&(df['ctrl_exp_forcedfp_pr']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) &
               (df['pre_experience'].isnull()==False)&(df['missing']==0)]
work_dum = pd.get_dummies(df['work_category']).rename(columns=lambda x: 'work_dum_' + str(x))
year_dum = pd.get_dummies(df['year']).rename(columns=lambda x: 'year_dum_' + str(x))
df = pd.concat([df,work_dum],axis = 1)
df = pd.concat([df,year_dum],axis = 1)

work_list = df['work_category'].unique()
year_list = df['year'].unique()

outcome = ['discount', 'delay_ratio', 'overrun_ratio', 'days_to_award']

#iteration
for o in outcome:

    #make a column list
    reg_col = []
    for i in work_list:
        reg_col.append('work_dum_' + i)
    for j in year_list:
        reg_col.append('year_dum_'+ str(j))
    exog_var = ['fpsb_auction','reserve_price','municipality']
    exog = exog_var + reg_col 


    #exog.remove('year_dum_2000.0')
    #exog.remove('work_dum_OG01')
    #exog.remove('municipality')
    #exog.remove('reserve_price')
    
    #exog.remove('work_dum_OS03')
    #exog.remove('work_dum_OS04')
    #exog.remove('work_dum_OS33')
    #exog.remove('work_dum_OG13')
    #exog.remove('work_dum_OS09')
    #exog.remove('work_dum_OS13')
    #exog.remove('work_dum_OS32')
    #exog.remove('work_dum_OG09')
    #exog.remove('work_dum_OG21')
    #exog.remove('work_dum_OG18')
    #exog.remove('work_dum_OG24')
    #exog.remove('work_dum_OG05')
    #exog.remove('year_dum_2008.0')
    #exog.remove('year_dum_2009.0')
    #exog.remove('year_dum_2007.0')
    #exog.remove('year_dum_2010.0')
    
    #check multicollinearity
    if o == 'discount':
        exog.remove('municipality')
        exog.remove('reserve_price')
        exog.remove('work_dum_OS26')
        exog.remove('year_dum_2004.0')
        exog.remove('work_dum_OG02')
        exog.remove('work_dum_OG01')
        exog.remove('work_dum_OG03')
        exog.remove('work_dum_OS28')
        exog.remove('work_dum_OS30')
        fe_reg_discount = mt.reg(df, o, exog, fe_name = 'authority_code', cluster = 'auth_anno',addcons= True, check_colinear = True)

    elif o == 'delay_ratio':
        fe_reg_delay = mt.reg(df, o, exog, fe_name = 'authority_code', cluster = 'auth_anno',addcons = True, check_colinear =  True)

    elif o == 'overrun_ratio':
        fe_reg_overrun = mt.reg(df, o, exog, fe_name = 'authority_code', cluster = 'auth_anno',addcons = True, check_colinear = True)

    else :
        fe_reg_award = mt.reg(df, o, exog, fe_name = 'authority_code', cluster = 'auth_anno',addcons = True, check_colinear = True)


fe_reg = (fe_reg_discount, fe_reg_delay, fe_reg_overrun, fe_reg_award )

ValueError: Colinear variables: 
work_dum_OS06

In [241]:
fe_reg

(=======================================================
 Dependent variable:	discount
 N:			1208
 R-squared:		0.1345
 Estimation method:	OLS
 VCE method:		Cluster
   Cluster variable:	  auth_anno
   No. of clusters:	  113
 Fixed effects by:	authority_code
   No. of FE:		  17
               coeff    se     t   p>t CI_low CI_high
 reserve_price 0.000 0.000 7.097 0.000  0.000   0.000
 =======================================================,
 Dependent variable:	delay_ratio
 N:			1072
 R-squared:		0.0889
 Estimation method:	OLS
 VCE method:		Cluster
   Cluster variable:	  auth_anno
   No. of clusters:	  110
 Fixed effects by:	authority_code
   No. of FE:		  17
                coeff    se      t   p>t CI_low CI_high
 reserve_price -0.000 0.000 -2.829 0.006 -0.000  -0.000
 =======================================================,
 Dependent variable:	overrun_ratio
 N:			1060
 R-squared:		0.1273
 Estimation method:	OLS
 VCE method:		Cluster
   Cluster variable:	  auth_anno
   No. of clusters: